***
## ASFSession

Because downloading any product in the ASF archive requires authentication, `asf_search` provides the `ASFSession` class, a subclass of `Session` with a few specific methods added to make authentication straightforward.

Using .netrc credentials is the preferred method for authentication. For more information, see the [session authentication documentation](https://docs.asf.alaska.edu/asf_search/downloading/#session-authentication)

A new, unauthenticated session can be created, although the authentication methods listed below allow chaining for direct creation of an authenticated session.

In [1]:
rslc_1 = 's3://nisar-st-data-ondemand/ADT_Testing/output/0.h5' # string
rslc_2 = 's3://nisar-st-data-ondemand/ADT_Testing/output/1.h5' # string
dem = 'url' # string
dem_params = '-b -123 35 -121 38' # string
gpu_enabled = False # boolean
s3_upload = False # boolean
s3_url = 's3://nisar-st-data-ondemand/ADT_Testing/' # string
key = '' # string
secret = '' # string
token = '' # string
region = '' # string

# hysds specifications
_time_limit = 3600
_soft_time_limit = 3600
_disk_usage = "20GB"
_submission_type = "iteration"
_label = "RSLC to INSAR PGE"

### `auth_with_creds()`
This authentication method accepts a username and password and establishes an authentication session with EDL and ASF.

In [2]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build/install')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/focus.yaml'), 'r') as f:
    FOCUS_YML = yaml.safe_load(f)

with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/insar.yaml'), 'r') as f:
    INSAR_YML = yaml.safe_load(f)

def download_alos_data(urls: list):
    """Downloads ALOS-1 data given by two urls and returns the paths to the ."""
    try:
        current_dir = os.getcwd()
        user_pass_session = asf.ASFSession().auth_with_creds(username, password)
        asf.download_urls(urls=urls, path=DOWNLOAD_DIR, session=user_pass_session, processes=2)
        files = os.listdir(DOWNLOAD_DIR)
        os.chdir(EXTRACT_DIR)
        extract_dirs = []
        for f in files:
            splitext = os.path.splitext(f)
            zip_f = os.path.join(DOWNLOAD_DIR, f)
            !unzip -o {zip_f}
            extract_dirs.append(os.path.abspath(splitext[0]))
        os.chdir(current_dir)
        print('Extracted:', extract_dirs)
        return extract_dirs
    except asf.ASFAuthenticationError as e:
        print(f'Auth failed: {e}')

def write_focus_config(target_path: str, yml_path: str):
    """Writes a focus.py runconfig with the specified target path."""
    FOCUS_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    FOCUS_YML['runconfig']['groups']['input_file_group']['input_file_path'] = [target_path]
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_output_file'] = os.path.join('RSLC', os.path.basename(target_path))
    FOCUS_YML['runconfig']['groups']['product_path_group']['sas_config_file'] = yml_path
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(FOCUS_YML, f, default_flow_style=False)

def write_insar_config(f1: str, f2: str, yml_path: str):
    """Writes the INSAR runconfig with the two specified RSLCs."""
    INSAR_YML['runconfig']['groups']['worker']['gpu_enabled'] = gpu_enabled
    INSAR_YML['runconfig']['groups']['input_file_group']['reference_rslc_file'] = f1
    INSAR_YML['runconfig']['groups']['input_file_group']['secondary_rslc_file'] = f2
    with open(yml_path, 'w', encoding='utf-8') as f:
        yaml.dump(INSAR_YML, f, default_flow_style=False)

print(WORKING_DIR)

/home/jovyan/Github/alos-to-insar/notebook_pges


In [ ]:
# Download the RSLCs from S3
dest_1 = aws_uploader.AWS.download_s3(rslc_1, os.path.join(DOWNLOAD_DIR, 'rslc_1.h5'))
dest_2 = aws_uploader.AWS.download_s3(rslc_2, os.path.join(DOWNLOAD_DIR, 'rslc_2.h5'))

# Download the DEM locally, which will appear as dem_0.tiff
!mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_dem.py {dem_params}

# Run INSAR
insar_run_config = os.path.join(HOME_DIR, 'insar_final.yml')
write_insar_config(dest_1, dest_2, insar_run_config)
!mamba run -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/insar.py --no-log {insar_run_config}

In [ ]:
from urllib.parse import urlparse

if s3_upload:
    parsed_url = urlparse(s3_url)
    bucket = parsed_url.netloc
    path = parsed_url.path

    # Upload to s3 bucket
    if key != '' or secret != '' or token != '' or region != '':
        # short term access key
        uploader = aws_uploader.AWS(key, secret, token, region)
    else:
        # uses role based access
        uploader = aws_uploader.AWS('', '', '', '', configdir=False)
    uploader.upload_dir(OUTPUT_DIR, bucket, path)
else:
    print('Flag for manual S3 upload was not set, skipping this step...')